# Starting with a real robot

The BotOp interface should work equally for a simulated and real robot. Note that currently only a locally compiled robotic package supports connecting to a Franka robot and Realsense camera. Here some first cautious steps to get started with a real robot.

## First robot motion, camera & pcl, and gripper motion

In [1]:
from robotic import ry
import numpy as np
import time
print(ry.compiled())

compile time: Sep 23 2023 18:22:45


In [2]:
#in case you switch to simulation
ry.params_add({'botsim/verbose': 1., 'physx/motorKp': 10000., 'physx/motorKd': 1000.})
ry.params_print()

-- ry.cpp:operator():99(0) python,
message: "Hello, the local 'rai.cfg' was loaded",
botsim/verbose: 1,
physx/motorKp: 10000,
physx/motorKd: 1000


First, also for a real robot we first load the configuration and maintain our own workspace configuration.

In [3]:
C = ry.Config()
C.addFile(ry.raiPath('../rai-robotModels/scenarios/pandaSingle.g'))
pcl = C.addFrame('pcl', 'cameraWrist')
C.view(False, 'this is your workspace data structure C -- NOT THE SIMULTATION')

0

Now we start botop with real=True. By passing the model configuration C, the system knows which and how many robots there should be an tries to connect with them.

In [4]:
# True = real robot!!
bot = ry.BotOp(C, useRealRobot=True)

-- bot.cpp:BotOp:38(0) CONNECTING TO FRANKAS..
STACK9 /usr/bin/python3(PyObject_Call
STACK8 /usr/bin/python3() [0x50b8a8]
STACK7 /usr/bin/python3(_PyObject_MakeTpCall
STACK6 /usr/bin/python3(PyCFunction_Call
STACK5 /home/mtoussai/.local/lib/python3.8/site-packages/robotic/ry.cpython-38-x86_64-linux-gnu.so(+0xbc6af) [0x7f9a446ad6af]
STACK4 /home/mtoussai/.local/lib/python3.8/site-packages/robotic/ry.cpython-38-x86_64-linux-gnu.so(+0x3bb9e) [0x7f9a4462cb9e]
STACK3 BotOp::BotOp(rai::Configuration&, bool)
STACK2 std::__shared_count<(__gnu_cxx::_Lock_policy)2>::__shared_count<FrankaThread, std::allocator<FrankaThread>, int, rai::Array<unsigned int>, Var<rai::CtrlCmdMsg>&, Var<rai::CtrlStateMsg>&>(FrankaThread*&, std::_Sp_alloc_shared_tag<std::allocator<FrankaThread> >, int&&, rai::Array<unsigned int>&&, Var<rai::CtrlCmdMsg>&, Var<rai::CtrlStateMsg>&)
STACK1 FrankaThread::init(unsigned int, rai::Array<unsigned int> const&)
STACK0 rai::LogToken::~LogToken()
-- WARNING:bot.cpp:BotOp:70(-1) Sta

== ERROR:franka.cpp:init:356(-2) not implemented with this compiler options: usually this means that the implementation needs an external library and a corresponding compiler option - see the source code


If that failed, you could `ry.params_print()` to see which global parameters were used and whether you should change them.

In [5]:
if bot.get_t()==0: #if the above failed, use a sim...
    del bot
    bot = ry.BotOp(C, useRealRobot=False)

-- bot.cpp:~BotOp:112(0) shutting down BotOp...


A first **motion**:

In [6]:
q = bot.get_qHome()
q[1] = q[1] + .2

bot.moveTo(q)
bot.wait(C)
print('first motion done')

bot.moveTo(bot.get_qHome())
bot.wait(C)

first motion done


0

Grabbing a **camera image & pcl**, adding the pcl to the workspace C:

In [7]:
pcl = C.addFrame('pcl', 'cameraWrist')
pcl.setShape(ry.ST.pointCloud, [2]) # the size here is pixel size for display
bot.sync(C)

while bot.getKeyPressed()!=ord('q'):
    image, depth, points = bot.getImageDepthPcl("cameraWrist")
    pcl.setPointCloud(points, image)
    pcl.setColor([1,0,0])
    bot.sync(C, .1)

-- WARNING:kin.cpp:addFrame:191(-1) frame already exists! returning existing without modifications!
-- frame.cpp:glDraw:1668(1) trying to draw empty mesh (shape type:pointCloud)
STACK9 /usr/bin/python3(PyCFunction_Call
STACK8 /home/mtoussai/.local/lib/python3.8/site-packages/robotic/ry.cpython-38-x86_64-linux-gnu.so(+0xbc6af) [0x7f9a446ad6af]
STACK7 /home/mtoussai/.local/lib/python3.8/site-packages/robotic/ry.cpython-38-x86_64-linux-gnu.so(+0x47f29) [0x7f9a44638f29]
STACK6 BotOp::getImageDepthPcl(rai::Array<unsigned char>&, rai::Array<float>&, rai::ArrayDouble&, char const*, bool)
STACK5 rai::Simulation::getImageAndDepth(rai::Array<unsigned char>&, rai::Array<float>&)
STACK4 rai::CameraView::updateConfiguration(rai::Configuration const&)
STACK3 rai::Configuration::getFrameState(rai::Array<rai::Frame*> const&) const
STACK2 rai::Frame::ensure_X()
STACK1 rai::Frame::calc_X_from_parent()
STACK0 rai::LogToken::~LogToken()


== ERROR:frame.cpp:calc_X_from_parent:94(-2) CHECK failed: 'parent->_state_X_isGood' -- 


RuntimeError: frame.cpp:calc_X_from_parent:94(-2) CHECK failed: 'parent->_state_X_isGood' -- 

Closing & opening the **gripper**:

In [ ]:
#slow close
bot.gripperMove(ry._left, width=.01, speed=.2)

while not bot.gripperDone(ry._left):
    bot.sync(C)

#fast open
bot.gripperMove(ry._left, width=.08, speed=1.)

while not bot.gripperDone(ry._left):
    bot.sync(C)

Always shut down the robot properly by destroying the handle:

In [8]:
del bot
del C

-- bot.cpp:~BotOp:112(0) shutting down BotOp...
-- simulation.cpp:~BotThreadedSim:56(0) shutting down SimThread
-- simulation.cpp:~Simulation:148(0) shutting down Simulation


## Advanced: Compliance & Force/Torque feedback

In [1]:
from robotic import ry
import numpy as np
import time

In [2]:
C = ry.Config()
C.addFile(ry.raiPath('scenarios/pandaSingle.g'))
C.view(False, 'this is your workspace data structure C -- NOT THE SIMULTATION')

0

In [3]:
# True = real robot!!
bot = ry.BotOp(C, False)

After opening the robot, it is floating (=setting the reference always to q_current) but damped (with D-gains). You should be able to move the robot now!

In [4]:
C.view(True, 'Try moving the robot by hand!\n-- press any key to continue --')

86

We can also turn off the damping, which makes the robot move more freely:

In [5]:
bot.hold(floating=True, damping=False)
C.view(True, 'Try moving the robot by hand!\n-- press any key to continue --')

107

Now let's hold stiffly around a constant reference:

In [6]:
bot.hold(floating=False, damping=True)
C.view(True, 'Try moving the robot by hand!\n-- press any key to continue --')

111

The `hold` methods above might be useful for kinestetic teaching or so (you can always keep C sync'ed and compute any features while moving the robot).

But for autonomous compliant manipulation we want to follow a reference and impose compliance in following this reference *along some task space dimensions only*. I.e., a task space compliance.

In [7]:
bot.moveTo(bot.get_qHome(), 1.)
bot.wait(C)

while True:
    bot.sync(C, .1)
    if bot.getKeyPressed()==ord('q'):
       break
    y, J = C.eval(ry.FS.position, ["l_gripper"], [[1,0,0]])
    bot.setCompliance(J, 1.)
    print(' direct:', J @ bot.get_tauExternal(),
          ' pseudoInv:', np.linalg.pinv(J.T) @ bot.get_tauExternal())

bot.setCompliance([], 0.)

 direct: [nan]  pseudoInv: [nan]
 direct: [nan]  pseudoInv: [nan]
 direct: [nan]  pseudoInv: [nan]
 direct: [nan]  pseudoInv: [nan]
 direct: [nan]  pseudoInv: [nan]
 direct: [nan]  pseudoInv: [nan]
 direct: [nan]  pseudoInv: [nan]
 direct: [nan]  pseudoInv: [nan]
 direct: [nan]  pseudoInv: [nan]
 direct: [nan]  pseudoInv: [nan]
 direct: [nan]  pseudoInv: [nan]
 direct: [nan]  pseudoInv: [nan]
 direct: [nan]  pseudoInv: [nan]
 direct: [nan]  pseudoInv: [nan]
 direct: [nan]  pseudoInv: [nan]
 direct: [nan]  pseudoInv: [nan]
 direct: [nan]  pseudoInv: [nan]
 direct: [nan]  pseudoInv: [nan]
 direct: [nan]  pseudoInv: [nan]
 direct: [nan]  pseudoInv: [nan]
 direct: [nan]  pseudoInv: [nan]
 direct: [nan]  pseudoInv: [nan]
 direct: [nan]  pseudoInv: [nan]
 direct: [nan]  pseudoInv: [nan]
 direct: [nan]  pseudoInv: [nan]
 direct: [nan]  pseudoInv: [nan]
 direct: [nan]  pseudoInv: [nan]
 direct: [nan]  pseudoInv: [nan]
 direct: [nan]  pseudoInv: [nan]
 direct: [nan]  pseudoInv: [nan]
 direct: [

In [ ]:
bot.home(C)

In [ ]:
del bot